In [12]:
import pandas as pd
import numpy as np

testCSV = pd.read_csv('jugaad(processed lyrics).csv')

#Baby by Justin Bieber (shitty popularity)
#userLyrics = "Ooh whoa, ooh whoa, ooh whoa You know you love me, I know you care Just shout whenever and I'll be there You are my love, you are my heart And we will never, ever, ever be apart Are we an item? Girl quit playin' We're just friends, what are you sayin' Said there's another, look right in my eyes My first love, broke my heart for the first time"

# def basicLyricsProcessing(userLyrics):
    

from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000,
                      lowercase=True,
                      ngram_range=(1,1),
                      analyzer = "word").fit(testCSV['Lyrics'].values.astype(str))
len(bow.vocabulary_)


lyrics_bow = bow.transform(testCSV['Lyrics'].values.astype(str))
print('Shape of Sparse Matrix: ', lyrics_bow.shape)
lyrics_bow.nnz

df= pd.DataFrame()
df['lyrics']=list(lyrics_bow.toarray())
df['hit'] =  testCSV['Hit']

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(df['lyrics'].tolist(), df['hit'].tolist())




Shape of Sparse Matrix:  (703, 1000)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
def predict():
    userLyrics = lyricsTextBox.get(1.0,"end-1c")
    #Everything in lowercase
    lowerCase = lambda x: " ".join(x.lower() for x in str(x).split())
    userLyrics = lowerCase(userLyrics)

    #Removing punctuation that does not add meaning to the song
    userLyrics = userLyrics.replace('[^\w\s]','')
    
    #Removing of stop words
    from nltk.corpus import stopwords

    stop = stopwords.words('english')
    removeStopWords = lambda x: " ".join(x for x in str(x).split() if x not in stop)
    userLyrics = removeStopWords(userLyrics)
    
    #Correction of Spelling mistakes
    from textblob import TextBlob
    spellingMistake = lambda x: str(TextBlob(x).correct())
    userLyrics = spellingMistake(userLyrics)
    
    #Lemmatization is basically converting a word into its root word. It is preferred over Stemming.
    from textblob import Word
    lemmatize = lambda x: " ".join([Word(word).lemmatize() for word in x.split()])
    userLyrics = lemmatize(userLyrics)
    
    #CountVectorization of user lyrics
    from sklearn.feature_extraction.text import CountVectorizer
    user_bow = CountVectorizer(max_features=1000,
                          lowercase=True,
                          ngram_range=(1,1),
                          analyzer = "word").fit([userLyrics])
    
    #Bag of Words conversion of user lyrics
    user_lyrics_bow = bow.transform([userLyrics])
    
    #Tf-idf transforming of user lyrics
    from sklearn.feature_extraction.text import TfidfTransformer
    user_tfidf_transformer = TfidfTransformer().fit(user_lyrics_bow)
    user_lyrics_tfidf = user_tfidf_transformer.transform(user_lyrics_bow)
    df_user = pd.DataFrame()
    df_user['lyrics']=list(user_lyrics_tfidf.toarray())


    user_prediction = model.predict(df_user['lyrics'].tolist())
    if(user_prediction[0] == 1):
        resultLabel.config(text = 'Song is Hit')
    else:
        resultLabel.config(text = 'Song is not Hit')


In [14]:
import tkinter as tk
import tkinter.ttk as ttk



# creating root
root = tk.Tk()
root.geometry("600x500")
rows = 0

while rows < 2:
    root.rowconfigure(rows, weight=1)
    root.columnconfigure(rows, weight=1)
    rows += 1

# creating a frame for storing all widgets
AppFrame = tk.Frame(root)
AppFrame.grid(row=0, column=0)

# Creating label for title
AppTitle = tk.Label(AppFrame, text="Hit Predictor 5000", font=("Arial", 30))
AppTitle.grid(row=0, column=0)
AppTitle2 = tk.Label(AppFrame, text="Patent Pending", font=("Arial", 5))
AppTitle2.grid(row=0, column=1)

''' Creating the first tab of the application... this tab is for the songwriter.'''
# Creating tabs for 2 use cases
notebook = ttk.Notebook(AppFrame)
notebook.grid(row=2, column=0, sticky="W", rowspan=100, columnspan=230)

# Define our first Tab. This tab contains textbox for
# entering lyrics and button for applying algorithm.
page1 = ttk.Frame(root)
notebook.add(page1, text='Lyrics Analysis')

# Label above textbox to tell user to "Enter lyrics Here".
EnterLyricsHereLabel = ttk.Label(page1, text="Enter Lyrics here:", padding=5)
EnterLyricsHereLabel.grid(row=1, column=0)

# Now create textbox in this "Lyrics Analysis" tab. Lyrics are input in this textbox
lyricsTextBox = tk.Text(page1, height=20, width=50)
lyricsTextBox.grid(row=2, column=0)# , rowspan=50, columnspan=50)


# Button for Applying processing and prediciton to Lyrics.
processButton = ttk.Button(page1, text="Apply Processing", width=40, command=predict)
processButton.grid(row=3, column=0)

# Label for showcasing the result.
resultLabel = ttk.Label(page1, text="\"Result will be shown here inplace of this text\"",
                             font=("Arial", 20))
resultLabel.grid(row=5, column=0)

'''  This part of code now declares the page2, use case of music production team.
'''

''' Creating page2 for Music Production company use cases features'''
page2 = ttk.Frame(root)
notebook.add(page2, text='Music Extras')

# main loop the root window
root.mainloop()




Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\moksh\Anaconda3\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-13-fc2c91d75e94>", line 32, in predict
    analyzer = "word").fit([userLyrics])
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py", line 836, in fit
    self.fit_transform(raw_documents)
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py", line 869, in fit_transform
    self.fixed_vocabulary_)
  File "C:\Users\moksh\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py", line 811, in _count_vocab
    raise ValueError("empty vocabulary; perhaps the documents only"
ValueError: empty vocabulary; perhaps the documents only contain stop words
